In [1]:
!pip install bitstring

In [2]:
from bitstring import BitArray
import numpy as np
import random
import json

In [3]:
'''
3x3 dots and boxes 
4 boxes 
12 lines to choose from for each move
2 power 12 game states for the agent to learn from
DRAW / TIE is available in this version of the game
Reward 100 and Penalty -100
'''
QTable=dict()
VISIT_TABLE=dict()
DISCOUNT_RATE=0.8
LEARNING_RATE=0.2

In [4]:
def learner(gameCount=1000):
    initialiseQTable()
    count=1
    choiceOfMove=['random','simple','qlearner']
    while count<gameCount:
        print("GAME ",count)
        if count<1000:
            playGame(choiceOfMove[0],False)
        elif count<5000:
            playGame(choiceOfMove[1],False)
        else:
            playGame(choiceOfMove[2],False)
        count+=1
        with open('Qtable.json','w') as jsonfile:
          json.dump(QTable,jsonfile)



    
        
        

In [5]:

def initialiseQTable():
    global QTable
    global VISIT_TABLE
    for i in range(4096):
        currentState=BitArray(uint=i, length=12)
        actionQvalues=dict()
        # find possible actions
        bitString=currentState.bin
        for index in range(len(bitString)):
            if bitString[index]=='0':
                actionQvalues[index+1]=0
        QTable[currentState.bin]=actionQvalues
        VISIT_TABLE[currentState.bin]=0
        QTable['111111111111']={0:0}
initialiseQTable()

                
        
    

In [6]:
def bestActionForState(state):
    maxQval=-1 #negative inf
    bestAction=0
    for action,qval in QTable[state].items():
        if qval>maxQval:
            maxQval=qval
            bestAction=action
    return bestAction




def updateQTable(memoryOfPlayerMoves):
    global QTable
    memoryOfPlayerMoves=memoryOfPlayerMoves[::-1] #reverse chronological order
    for state,action,nextState,reward in memoryOfPlayerMoves:
        maxQValForNextState=QTable[nextState][bestActionForState(nextState)]
        existingValue=(1-LEARNING_RATE)*QTable[state][action]
        updateValue=LEARNING_RATE*(reward+DISCOUNT_RATE*maxQValForNextState)
        QTable[state][action]=existingValue+updateValue
        VISIT_TABLE[state]+=1    
    

In [7]:
def printTable(currentState):
    count=1
    m=7
    for i in range(3):
        for j in range(3):
            print(".\t", end=" ")
            if j<2:
                if currentState[count-1]=='0':
                    print(str(count)+"\t",end=" ") 
                else:
                    print("--\t",end=" ")
                count+=1
                
        print("\n")
        if i!=2:
            for k in range(3):
                if currentState[m-1]=='0':
                    print(str(m)+"\t\t",end=" ")
                else:
                    print("|\t\t",end=" ")
                m+=1
        print("\n")
# printTable('111111111111')

In [8]:

def MakeMove(currentState,choice):
    # possible actions
    possibleActions=list(QTable[currentState].keys())
#     print(possibleActions)
    #choiceOfMove=['random','simple','qlearner']
    if choice=='random':
        return possibleActions[random.randint(0,len(possibleActions)-1)]
        
    if choice=='simple':
        minCount=1000000
        action=0
        for action in possibleActions:
            if VISIT_TABLE[transition(currentState,action)]<minCount:
                minCount=VISIT_TABLE[transition(currentState,action)]
                bestAction=action     
        return bestAction
    if choice=='qlearner':
        return bestActionForState(currentState)  

    
 

In [9]:
def isBoardFinished(boardState):
    return boardState.bin=='111111111111'

In [10]:
def transition(currentState,currentMove):
    temp=currentState
    temp=temp[:currentMove-1]+'1'+temp[currentMove:]
#     print("Transition from state",currentState,"Is ",temp)
    return temp

#print(transition(BitArray(uint=10,length=12),11))

In [11]:
def numberOfNewBoxFormed(box,newState):
    count=0
    boxBounds=[[1,3,7,8],[2,4,8,9],[3,5,10,11],[4,6,11,12]]
    for i in range(len(boxBounds)):
      if box[i]==0:
        flag=True
        for lineNumber in boxBounds[i]:
          if newState[lineNumber-1]!='1':
            flag=False
            break
        if flag==True:
          count+=1
          box[i]=1
    return count
        
                

In [12]:
def playGame(choiceOfMove,withHuman=False):
    #Game init
    player=["p1","p2"]
    if withHuman:
        player[1]='human'
    currentPlayer="p1"
    box=[0,0,0,0]
    currentState=BitArray(uint=0, length=12)
    memory={player[0]:[],player[1]:[]}
    score={player[0]:0,player[1]:0}
    while isBoardFinished(currentState)==False:
        # print("Playing game")
        if(currentPlayer!="human"):
            currentMove=MakeMove(currentState.bin,choiceOfMove)
        else:
          possibleActions=list(QTable[currentState.bin].keys())
          currentMove=int(input("Enter the line number:"))
          while currentMove not in possibleActions:
            currentMove=int(input("INVALID!Enter the line number:"))

        newState=transition(currentState.bin,currentMove) 
        memory[currentPlayer].append([currentState.bin,currentMove,newState,0])
        numberOfBoxes=numberOfNewBoxFormed(box,newState)
        if numberOfBoxes>0:#Update this......:)
            score[currentPlayer]+=numberOfBoxes*2
            memory[currentPlayer][:-1][0][-1]=100
            memory[player[1-player.index(currentPlayer)]][:-1][0][-1]=-100

        else:
            currentPlayer=player[1-player.index(currentPlayer)]

        currentState=BitArray(uint=int(newState,2),length=12)
        if player[1]=='human':
            
            printTable(currentState.bin)
            print("Score of ",player[0]," is ",score[player[0]])
            print("Score of ",player[1]," is ",score[player[1]])
            if isBoardFinished(currentState)==False:
              print("Next turn for ",currentPlayer)

    #find winner 
    result=""
    if score[player[0]]==score[player[1]]:
        result='TIE'
        print(result)
    else:
        result=max(score,key=score.get)
        print("Winner is ",result)
    #print(memory[player[0]])
    if result=='TIE':
        memory[player[0]][:-1][0][-1]=50
        memory[player[1]][:-1][0][-1]=50
    else: 
        memory[result][:-1][0][-1]=100
        memory[player[1-player.index(result)]][:-1][0][-1]=-100    
    #put reward and penalties in last row of the memory table
    updateQTable(memory[player[0]])
    updateQTable(memory[player[1]])





In [ ]:
# initialiseQTable()
learner(gameCount=1000)
playGame(choiceOfMove='qlearner',withHuman=True)

GAME  1
Winner is  p1
GAME  2
Winner is  p2
GAME  3
Winner is  p2
GAME  4
Winner is  p2
GAME  5
TIE
GAME  6
Winner is  p1
GAME  7
Winner is  p1
GAME  8
Winner is  p2
GAME  9
Winner is  p2
GAME  10
Winner is  p2
GAME  11
Winner is  p2
GAME  12
TIE
GAME  13
Winner is  p2
GAME  14
Winner is  p2
GAME  15
Winner is  p1
GAME  16
Winner is  p1
GAME  17
Winner is  p1
GAME  18
Winner is  p1
GAME  19
Winner is  p2
GAME  20
Winner is  p2
GAME  21
Winner is  p1
GAME  22
Winner is  p1
GAME  23
Winner is  p2
GAME  24
Winner is  p2
GAME  25
Winner is  p1
GAME  26
TIE
GAME  27
Winner is  p1
GAME  28
Winner is  p2
GAME  29
Winner is  p2
GAME  30
Winner is  p2
GAME  31
Winner is  p2
GAME  32
Winner is  p1
GAME  33
Winner is  p1
GAME  34
Winner is  p1
GAME  35
Winner is  p1
GAME  36
Winner is  p1
GAME  37
Winner is  p2
GAME  38
Winner is  p2
GAME  39
TIE
GAME  40
Winner is  p2
GAME  41
Winner is  p1
GAME  42
TIE
GAME  43
Winner is  p2
GAME  44
Winner is  p1
GAME  45
Winner is  p2
GAME  46
Winner is  p2
G

In [ ]:
# playGame(choiceOfMove='qlearner',withHuman=True)

with open('Qtable.json','w') as jsonfile:
  json.dump(QTable,jsonfile)

print(VISIT_TABLE)